## ANÁLISE DO CLUSTER 0 DA UP GLÓRIA

In [1]:
# Bibliotecas
import numpy as np
import pandas as pd

In [2]:
# Upload do arquivo para análise
gloria_0_caminho = r"C:\Users\Administrador\Downloads\TCC-20240110T120732Z-001\Dados_UP Gloria\Cluster 0\Cluster0_GLORIA_ALTERACOES.csv"
gloria_0 = pd.read_csv(gloria_0_caminho, sep = ";", encoding= "utf8" )
gloria_0

,Linha,Sublinha,PC,SIU,id_hex,mes,ano,Alteracao_QH,Alteracao_IT,pass,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao,Cluster
0,1404C,1,1,jul/11,89a88cdaadbffff,AGO,2022,1,0,4,0,0,0,0
1,1404C,1,1,dez/10,89a88cdaac3ffff,AGO,2022,1,0,4,0,0,0,0
2,1404C,1,1,dez/10,89a88cdaac3ffff,AGO,2022,1,0,2,0,0,0,0
3,1404C,1,2,dez/10,89a88cdaac3ffff,AGO,2022,1,0,30,0,0,0,0
4,1404C,1,2,dez/10,89a88cdaac3ffff,AGO,2022,1,0,42,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,4802A,12,1,41495,89a88cdae4fffff,AGO,2023,0,0,0,0,0,0,0
2660,4802A,12,1,40315,89a88cdae4fffff,AGO,2023,0,0,0,0,0,0,0
2661,4802A,12,1,41375,89a88cd85b3ffff,AGO,2023,0,0,0,0,0,0,0
2662,4802A,12,1,40311,89a88cd85b3ffff,AGO,2023,0,0,0,0,0,0,0


# Estatísticas descritivas

In [3]:
estatisticas_descritivas = ['Alteracao_QH', 'Alteracao_IT', 'Houve aumento passageiro', 'Tarifa_Aumento', 'Tarifa_reducao']
for col in estatisticas_descritivas:
    gloria_0[col] = gloria_0[col].astype(int)

estatisticas = {}
for col in gloria_0.columns:
   estatisticas[col] = {
        'Total Observações': gloria_0[col].count(),
        '0 - Não': (gloria_0[col] == 0).sum(),
        '1 - Sim': (gloria_0[col] == 1).sum()
    }

estatisticas = pd.DataFrame(estatisticas)
estatisticas


,Linha,Sublinha,PC,SIU,id_hex,mes,ano,Alteracao_QH,Alteracao_IT,pass,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao,Cluster
Total Observações,2664,2664,2664,2664,2664,2664,2664,2664,2664,2664,2664,2664,2664,2664
0 - Não,0,0,0,0,0,0,0,1992,2611,1270,2040,2442,2442,2664
1 - Sim,0,1104,2148,0,0,0,0,672,53,113,624,222,222,0


# Teste V de Cramer
Objetivo: Testar asssociação entre as variáveis

In [4]:
# Pacotes
from scipy.stats import chi2_contingency

In [5]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [6]:
columns_to_test = ['Alteracao_QH', 'Alteracao_IT', 'Houve aumento passageiro', 'Tarifa_Aumento', 'Tarifa_reducao']
# Preparando um DataFrame para armazenar os resultados do V de Cramer
cramers_results_selected = pd.DataFrame(index=columns_to_test, columns=columns_to_test)

# Calculando o V de Cramer para cada par de variáveis selecionadas
for col1 in columns_to_test:
    for col2 in columns_to_test:
        cramers_results_selected.loc[col1, col2] = cramers_v(gloria_0[col1], gloria_0[col2])

cramers_results_selected

,Alteracao_QH,Alteracao_IT,Houve aumento passageiro,Tarifa_Aumento,Tarifa_reducao
Alteracao_QH,0.999005,0.191746,0.010947,0.172507,0.323145
Alteracao_IT,0.191746,0.990371,0.03342,0.032803,0.032803
Houve aumento passageiro,0.010947,0.03342,0.998953,0.0,0.092613
Tarifa_Aumento,0.172507,0.032803,0.0,0.997542,0.086319
Tarifa_reducao,0.323145,0.032803,0.092613,0.086319,0.997542


Análise: Associação fraca a moderada entre alterações no Quadro de Horário e redução da tarifa. Não há associação entre o aumento de passageiros e alterações realizadas pela SUMOB ou BHTRANS

# ANÁLISE BINOMIAL NEGATIVA

In [7]:
gloria0_cam = r"C:\Users\Administrador\Downloads\TCC-20240110T120732Z-001\Dados_UP Gloria\Cluster 0\CLUSTER0.xlsx"
gloria0 = pd.read_excel(gloria0_cam)
gloria0

,Hexagono,Bairro,UP,Mes,Aumentou,Quantidade_passageiros(Y),Numero_linhas,Frequencia_HEX,Populacao,Renda_DOMICILIAR,Quantidade_escolas,Cluster
0,89a88cd8537ffff,Glória,Glória,"agosto, 2022",0,61,3,100,1245,668.0,1,0
1,89a88cd8583ffff,Glória,Glória,"agosto, 2022",0,76,3,306,1703,627.0,1,0
2,89a88cd8593ffff,Álvaro Camargos,Glória,"agosto, 2022",0,17,1,52,2192,631.6,0,0
3,89a88cd85afffff,Glória,Glória,"agosto, 2022",0,8,3,94,1420,753.5,0,0
4,89a88cd85b3ffff,Novo Glória,Glória,"agosto, 2022",0,219,3,308,2021,725.7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
193,89a88cdae67ffff,Coqueiros,Glória,"agosto, 2023",1,619,1,76,1620,595.3,0,0
194,89a88cdae6bffff,Glória,Glória,"agosto, 2023",1,199,3,129,1245,707.7,1,0
195,89a88cdae73ffff,Pindorama,Glória,"agosto, 2023",1,157,2,81,2103,667.8,1,0
196,89a88cdae8bffff,Pindorama,Glória,"agosto, 2023",1,48,2,52,1475,415.4,0,0
